In [2]:
import pandas as pd
import pyarrow.feather as feather
import numpy as np
import matplotlib.pyplot as plt

In [137]:
crsp_policy_to_exclude = ['C & I', 'Bal', 'Bonds', 'Pfd', 'B & P', 'GS', 'MM', 'TFM']
lipper_class = ['EIEI', 'G', 'LCCE', 'LCGE', 'LCVE', 'MCCE', 'MCGE', 'MCVE',
                'MLCE', 'MLGE', 'MLVE', 'SCCE', 'SCGE', 'SCVE']
lipper_obj_cd = ['CA', 'EI', 'G', 'GI', 'MC', 'MR', 'SG']
si_obj_cd = ['AGG', 'GMC', 'GRI', 'GRO', 'ING', 'SCG']
wbrger_obj_cd=['G', 'G-I', 'AGG', 'GCI', 'GRI', 'GRO', 'LTG', 'MCG','SCG']
##############################
#
begin_date='1980-01-01'
end_date='2019-01-31'

##############################
with open('../../data/data/style.feather', 'rb') as f:
    style=feather.read_feather(f)
with open('../../data/data/names.feather', 'rb') as f:
    names=feather.read_feather(f)
with open('../../data/data/fund_summary.feather', 'rb') as f:
    fund_summary=feather.read_feather(f)
names.drop_duplicates(subset=['crsp_fundno'], keep='last', inplace=True)

style=style[(style['begdt'].astype(str)>=begin_date)&(style['enddt'].astype(str)<=end_date)]
names=names[(names['chgdt'].astype(str)>=begin_date)&(names['chgenddt'].astype(str)<=end_date)]
fund_summary=fund_summary[(fund_summary['caldt'].astype(str)>=begin_date)&(fund_summary['caldt'].astype(str)<=end_date)]

# investing on average less than 80% of their assets, excluding cash, in common stocks
fund_summary['per_cash']=fund_summary['per_cash'].replace(np.nan,0)
fund_summary['per_pref']=fund_summary['per_pref'].replace(np.nan,0)
fund_summary['per_eq_oth']=fund_summary['per_eq_oth'].replace(np.nan,0)
fund_summary['per_com']=fund_summary['per_com'].replace(np.nan,0)
fund_summary['per_com_st']=(fund_summary['per_com']+fund_summary['per_pref']+fund_summary['per_eq_oth'])*100/(100-fund_summary['per_cash'])
fund_summary['per_com']=fund_summary['per_com_st']

per_com = fund_summary.groupby(['crsp_fundno']).apply(
    lambda x: np.nansum(x['per_com']) / np.count_nonzero(x['per_com']))
per_com = per_com.reset_index()
per_com.columns = ['crsp_fundno', 'per_com']
style = pd.merge(style, per_com)
style = style[style['per_com'] > 80]

C:\Users\Justin\AppData\Local\Temp/ipykernel_24880/3084116576.py:34: RuntimeWarning: invalid value encountered in double_scalars
  lambda x: np.nansum(x['per_com']) / np.count_nonzero(x['per_com']))


In [136]:
funds2

,crsp_fundno,short_way
15,7.0,1
29,13.0,1
46,23.0,1
48,25.0,1
59,33.0,1
...,...,...
178747,94655.0,1
178761,94663.0,1
178780,94674.0,1
178793,94683.0,1


In [130]:
funds2

,crsp_fundno,short_way
15,7.0,1
29,13.0,1
46,23.0,1
48,25.0,1
59,33.0,1
...,...,...
178747,94655.0,1
178761,94663.0,1
178780,94674.0,1
178793,94683.0,1


In [138]:
funds = style[(style['lipper_class'].isin(lipper_class)) | (style['lipper_obj_cd'].isin(lipper_obj_cd)) | (
    style['si_obj_cd'].isin(si_obj_cd)) | (style['wbrger_obj_cd'].isin(wbrger_obj_cd))]
funds = funds[~funds['policy'].isin(crsp_policy_to_exclude)]
funds['long_way'] = 1
funds = funds[['crsp_fundno', 'long_way']]
funds.drop_duplicates(inplace=True)

In [139]:
funds2 = style[(style['crsp_obj_cd'].str[:1] == 'E') & (style['crsp_obj_cd'].str[1:2] == 'D') & (
    style['crsp_obj_cd'].str[2:3].isin(['C', 'Y'])) & (~style['crsp_obj_cd'].str[2:4].isin(['YH', 'YS'])) & (
                       style['si_obj_cd'] != 'OPI')]
funds2['short_way'] = 1
funds2 = funds2[['crsp_fundno', 'short_way']]
funds2.drop_duplicates(inplace=True)

C:\Users\Justin\AppData\Local\Temp/ipykernel_24880/2434221775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds2['short_way'] = 1


In [140]:
funds4 = pd.merge(funds2, style[['crsp_fundno', 'crsp_obj_cd', 'begdt']], on=['crsp_fundno'],
                  sort=['crsp_fundno', 'begdt'])
funds4['flipper'] = 0
funds4.loc[~funds4['crsp_obj_cd'].str[0:3].isin(['EDC', 'EDY']), 'flipper'] = 1
funds5 = funds4.groupby('crsp_fundno', group_keys=False).apply(lambda x: x.loc[x.flipper.idxmax()])
funds5.reset_index(drop=True, inplace=True)

funds6 = pd.merge(funds5, names, on='crsp_fundno')

counties = funds6.groupby('crsp_fundno')['index_fund_flag'].sum() == 0
funds7 = funds6.loc[counties.values == True]
funds7['namex'] = funds7['fund_name'].str.lower()

funds8 = funds7[(~funds7['namex'].str.contains('index')) & (~funds7['namex'].str.contains('s&p')) & (
    ~funds7['namex'].str.contains('idx')) &
                (~funds7['namex'].str.contains('dfa')) & (~funds7['namex'].str.contains('program')) & (
                    ~funds7['namex'].str.contains('etf')) &
                (~funds7['namex'].str.contains('exchange traded')) & (
                    ~funds7['namex'].str.contains('exchange-traded')) &
                (~funds7['namex'].str.contains('target')) & (~funds7['namex'].str.contains('2005')) &
                (~funds7['namex'].str.contains('2010')) & (~funds7['namex'].str.contains('2015')) &
                (~funds7['namex'].str.contains('2020')) & (~funds7['namex'].str.contains('2025')) &
                (~funds7['namex'].str.contains('2030')) & (~funds7['namex'].str.contains('2035')) &
                (~funds7['namex'].str.contains('2040')) & (~funds7['namex'].str.contains('2045')) &
                (~funds7['namex'].str.contains('2050')) & (~funds7['namex'].str.contains('2055'))]

with open('../../data/data/mflink.feather', 'rb') as f:
    mflink=feather.read_feather(f)

equity_funds = pd.merge(funds8, mflink, on='crsp_fundno')
equity_funds = equity_funds[equity_funds['flipper'] == 0]

C:\Users\Justin\AppData\Local\Temp/ipykernel_24880/79663366.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds7['namex'] = funds7['fund_name'].str.lower()


In [141]:
equity_funds

,crsp_fundno,short_way,crsp_obj_cd,begdt,flipper,cusip8,chgdt,chgenddt,crsp_portno,crsp_cl_grp,...,vau_fund,et_flag,delist_cd,header,first_offer_dt,end_dt,dead_flag,merge_fundno,namex,wficn
0,7.0,1,EDYG,1994-12-30,0,02317K67,1994-12-30,1995-05-31,NaN,NaN,...,N,None,M,1.0,1992-01-01,1995-05-31,Y,21296.0,ambassador funds:established co gr/fiduciary,100154.0
1,23.0,1,EDYB,1994-12-30,0,None,1994-01-01,1994-10-31,NaN,NaN,...,N,None,L,1.0,1994-01-01,1994-10-31,Y,NaN,alexander hamilton equity growth & income fd,100088.0
2,25.0,1,EDYG,1993-12-31,0,02317K65,1995-01-01,1995-05-31,NaN,NaN,...,N,None,M,1.0,1992-01-01,1995-05-31,Y,21286.0,ambassador funds:established co gr/retail a,100154.0
3,33.0,1,EDYG,1997-12-31,0,None,1999-12-31,1999-11-30,NaN,2000006.0,...,N,None,L,1.0,1994-12-09,1999-11-30,Y,NaN,professionally managed portfolios: academy val...,105402.0
4,43.0,1,EDYG,1991-12-31,0,None,1993-01-02,1994-11-30,NaN,NaN,...,N,None,M,1.0,1990-01-01,1994-11-30,Y,18088.0,advisors fund lp/a,100039.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,91662.0,1,EDCM,2017-08-03,0,54400A48,2017-08-08,2018-06-29,1027645.0,2005872.0,...,N,None,L,1.0,2017-07-28,2018-06-29,Y,NaN,lord abbett securities trust: lord abbett valu...,410559.0
7613,91717.0,1,EDYG,2017-08-14,0,36240150,2017-08-15,2018-08-31,1021754.0,2014540.0,...,N,None,L,1.0,2017-07-28,2018-08-31,Y,NaN,"gabelli esg fund, inc; class t shares",501719.0
7614,92632.0,1,EDCM,2017-11-27,0,14214L52,2018-04-04,2018-06-29,1024906.0,2004627.0,...,N,None,M,1.0,2017-11-20,2018-06-29,Y,92633.0,carillon series trust: carillon eagle mid cap ...,106353.0
7615,92635.0,1,EDCS,2017-11-27,0,14214L36,2018-04-04,2018-06-29,1029241.0,2013649.0,...,N,None,M,1.0,2017-11-20,2018-06-29,Y,92608.0,carillon series trust: carillon eagle smaller ...,601590.0


In [115]:
with open('../../data/data/tna_ret_nav.feather', 'rb') as f:
    tna_ret_nav=feather.read_feather(f)
tna_ret_nav=tna_ret_nav[(tna_ret_nav['caldt'].astype(str)>=begin_date)&(tna_ret_nav['caldt'].astype(str)<=end_date)]

returns1 = pd.merge(equity_funds, tna_ret_nav, on='crsp_fundno')
returns1.rename(columns={'caldt': 'date'}, inplace=True)

# compute gross return
with open('../../data/data/fund_fees.feather', 'rb') as f:
    fund_fees=feather.read_feather(f)
returns2 = pd.merge(returns1, fund_fees, on='crsp_fundno')
returns2 = returns2[(returns2['date'] >= returns2['begdt_y']) & (returns2['date'] <= returns2['enddt'])]
returns2['rret'] = returns2['mret'] + returns2['exp_ratio'] / 12
returns2.sort_values(by=['crsp_fundno', 'date'], inplace=True)

returns2['flag'] = (returns2.crsp_fundno != returns2.crsp_fundno.shift()).astype(int)
returns2['weight'] = returns2['mtna'].shift()
returns2.loc[returns2['flag'] == 1, 'weight'] = returns2.loc[returns2['flag'] == 1, 'mtna']
returns2 = returns2[returns2['weight'] != 0]
returns2['age'] = (returns2['end_dt'] - returns2['first_offer_dt']) / np.timedelta64(1, 'Y')

# keep only observations dated after the fund's first offer date.
returns2 = returns2[returns2['date'] > returns2['first_offer_dt']]

# aggregate multiple share classes
returns2.sort_values(by=['wficn', 'date'], inplace=True)

# monthly return
mret = returns2.groupby(['wficn', 'date']).apply(lambda x: np.average(x['mret'], weights=x['weight']))
returns = mret.reset_index()
returns.columns = ['wficn', 'date', 'mret']

# monthly total net assets
mtna = returns2.groupby(['wficn', 'date']).apply(lambda x: np.sum(x['mtna']))
mtna = mtna.reset_index()
returns['mtna'] = mtna[0]

# weight
weight = returns2.groupby(['wficn', 'date']).apply(lambda x: np.sum(x['weight']))
weight = weight.reset_index()
returns['weight'] = weight[0]

# gross return
rret = returns2.groupby(['wficn', 'date']).apply(lambda x: np.average(x['rret'], weights=x['weight']))
rret = rret.reset_index()
returns['rret'] = rret[0]

# turnover ratio
turnover = returns2.groupby(['wficn', 'date']).apply(lambda x: np.average(x['turn_ratio'], weights=x['weight']))
turnover = turnover.reset_index()
returns['turnover'] = turnover[0]

# expense ratio
expense = returns2.groupby(['wficn', 'date']).apply(lambda x: np.average(x['exp_ratio'], weights=x['weight']))
expense = expense.reset_index()
returns['exp_ratio'] = expense[0]

# fund age
age = returns2.groupby(['wficn', 'date']).apply(lambda x: np.max(x['age']))
age = age.reset_index()
returns['age'] = age[0]

# fund flow
returns['cum_ret'] = (1 + returns['mret']).rolling(12).apply(np.prod, raw=True) - 1
returns['flow'] = (returns['mtna'] - returns.groupby(['wficn'])['mtna'].shift(12) * returns['cum_ret']) / \
                  returns.groupby(['wficn'])['mtna'].shift(12)
returns.loc[returns['flow'] > 20, 'flow'] = np.nan

# fund vol
returns['vol'] = returns.groupby(['wficn'])['mret'].rolling(12).std().reset_index()['mret']

# exclude funds with less than 12 months of observations
obs = returns.groupby(['wficn'])['mtna'].count()
obs = obs.reset_index()
obs.columns = ['wficn', 'obs']
returns = pd.merge(returns, obs)
returns = returns[returns['obs'] >= 12]

# We also exclude fund observations before a fund passes the $5 million threshold for assets under management (AUM).
# All subsequent observations, including those that fall under the $5 million AUM threshold in the future, are included.

mtna_group = 5
returns['tna_ind'] = 0
for w in returns['wficn'].unique():
    for index, row in returns[returns['wficn'] == w].iterrows():
        if row['mtna'] < mtna_group:
            returns.at[index, 'tna_ind'] += 1
        else:
            break

returns.drop(index=returns[returns['tna_ind'] == 1].index, inplace=True)

########### 75% no less than 5 million
aum = returns.sort_values(by=['wficn', 'date'], ascending=False).groupby(['wficn']).head(12)
for index, row in aum.iterrows():
    if row['mtna'] < mtna_group:
        aum.at[index, 'tna_ind'] += 1
total_aum = aum.groupby(['wficn'])['tna_ind'].sum()

returns = returns[~returns['wficn'].isin(total_aum[total_aum > 3].index)]
#returns=returns[(returns['date'].astype(str)>=begin_date)&(returns['date'].astype(str)<=end_date)]

In [1]:
'''import statsmodels.api as sm
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.summary()'''

'import statsmodels.api as sm\nx = sm.add_constant(x)\nmodel = sm.OLS(y,x)\nresults = model.fit()\nresults.summary()'

In [117]:
returns['wficn'].unique()

array([100001., 100003., 100004., ..., 608105., 608107., 608350.])

In [29]:
tna_ret_nav

,caldt,crsp_fundno,mtna,mret,mnav
10,1997-01-31,1.0,0.100,NaN,NaN
11,1997-02-28,1.0,3.926,NaN,14.98
12,1997-03-31,1.0,12.081,-0.011789,14.72
13,1997-04-30,1.0,20.540,0.008417,14.76
14,1997-05-30,1.0,26.157,0.013523,14.87
...,...,...,...,...,...
362520,2018-11-30,98815.0,NaN,0.006696,9.02
362521,2018-12-31,98815.0,NaN,0.005508,8.86
362522,2019-01-31,98815.0,NaN,0.028217,9.11
369555,2018-12-31,99689.0,NaN,NaN,9.45


C:\Users\Justin\AppData\Local\Temp/ipykernel_42440/1321094398.py:71: RuntimeWarning: invalid value encountered in double_scalars
  lambda x: np.nansum(x['per_com']) / np.count_nonzero(x['per_com']))
C:\Users\Justin\AppData\Local\Temp/ipykernel_42440/1321094398.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds2['short_way'] = 1
C:\Users\Justin\AppData\Local\Temp/ipykernel_42440/1321094398.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds7['namex'] = funds7['fund_name'].str.lower()


In [41]:
with open('../../data/data/mflink.feather', 'rb') as f:
    mflink=feather.read_feather(f)

In [103]:
equity_funds = pd.merge(funds8, mflink, on='crsp_fundno')

In [104]:
equity_funds = pd.merge(funds8, mflink, on='crsp_fundno')
equity_funds = equity_funds[equity_funds['flipper'] == 0]

In [51]:
funds8['flipper']

0        0
1        1
2        0
3        0
4        0
        ..
13870    0
13871    0
13873    0
13874    0
13875    0
Name: flipper, Length: 11451, dtype: int64

In [59]:
equity_funds['wficn'].unique()

array([100154., 100088., 100041., ..., 608386., 606588., 608273.])

In [90]:
eq=pd.read_csv('../eq_fund.csv')

In [91]:
eq['crsp_fundno']=(eq['crsp_fundno'].astype(str)+eq['Unnamed: 2'].astype(str)).astype(int)

In [92]:
eq

,wficn,crsp_fundno,Unnamed: 2
0,301534,2,2
1,100154,7,7
2,100088,23,3
3,100154,25,5
4,105402,33,3
...,...,...,...
14994,606590,99554,4
14995,608273,99648,8
14996,607049,99710,0
14997,607049,99711,1


In [88]:
equity_funds

,crsp_fundno,long_way,crsp_obj_cd,begdt,flipper,cusip8,chgdt,chgenddt,crsp_portno,crsp_cl_grp,...,vau_fund,et_flag,delist_cd,header,first_offer_dt,end_dt,dead_flag,merge_fundno,namex,wficn
0,7.0,1,EDYG,1994-12-30,0,02317K67,1994-12-30,1995-05-31,NaN,NaN,...,N,None,M,1.0,1992-01-01,1995-05-31,Y,21296.0,ambassador funds:established co gr/fiduciary,100154.0
1,23.0,1,EDYB,1994-12-30,0,None,1994-01-01,1994-10-31,NaN,NaN,...,N,None,L,1.0,1994-01-01,1994-10-31,Y,NaN,alexander hamilton equity growth & income fd,100088.0
2,25.0,1,EDYG,1993-12-31,0,02317K65,1995-01-01,1995-05-31,NaN,NaN,...,N,None,M,1.0,1992-01-01,1995-05-31,Y,21286.0,ambassador funds:established co gr/retail a,100154.0
3,51.0,1,EDYB,1993-12-31,0,None,1995-12-31,1996-08-30,NaN,NaN,...,N,None,L,1.0,1993-01-01,1996-08-30,Y,NaN,advisors inner circle:a+p large cap value,100041.0
6,93.0,1,EDYG,1961-12-29,0,None,1993-01-02,1993-01-29,NaN,NaN,...,N,None,M,1.0,1958-01-01,1993-01-29,Y,16669.0,american investors growth fund,100222.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,99554.0,1,EDYG,2022-02-09,0,01878H72,2022-02-15,2022-06-30,1050055.0,2055948.0,...,N,None,None,1.0,2022-01-31,2022-06-30,N,NaN,"ab cap fund, inc: ab sustainable us thematic p...",606590.0
9924,99648.0,1,EDYG,2022-03-02,0,90386K88,2022-04-01,2022-06-30,1051887.0,2075545.0,...,N,None,None,1.0,2022-02-28,2022-06-30,N,NaN,ultimus managers trust: evolutionary tree inno...,608273.0
9925,99710.0,1,EDCM,2022-03-14,0,14064D54,2022-04-01,2022-06-30,1050299.0,2057565.0,...,N,None,None,1.0,2022-03-10,2022-06-30,N,NaN,capitol series trust: fuller & thaler behavior...,607049.0
9926,99711.0,1,EDCM,2022-03-14,0,14064D53,2022-04-01,2022-06-30,1050299.0,2057565.0,...,N,None,None,1.0,2022-03-10,2022-06-30,N,NaN,capitol series trust: fuller & thaler behavior...,607049.0


In [93]:
equity_funds = pd.merge(eq, mflink, on='crsp_fundno')

In [95]:
equity_funds=eq

In [96]:
eq

,wficn,crsp_fundno,Unnamed: 2
0,301534,2,2
1,100154,7,7
2,100088,23,3
3,100154,25,5
4,105402,33,3
...,...,...,...
14994,606590,99554,4
14995,608273,99648,8
14996,607049,99710,0
14997,607049,99711,1


C:\Users\Justin\AppData\Local\Temp/ipykernel_2892/1109061608.py:74: RuntimeWarning: invalid value encountered in double_scalars
  lambda x: np.nansum(x['per_com']) / np.count_nonzero(x['per_com']))
C:\Users\Justin\AppData\Local\Temp/ipykernel_2892/1109061608.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds2['short_way'] = 1


In [33]:
funds2

,crsp_fundno,short_way
15,7.0,1
29,13.0,1
46,23.0,1
48,25.0,1
86,43.0,1
...,...,...
185000,100032.0,1
185001,100033.0,1
185002,100034.0,1
185003,100035.0,1


In [27]:
tna_ret_nav[tna_ret_nav['caldt']]

0         1960-01-29
1         1960-02-29
2         1960-04-29
3         1960-06-30
4         1960-07-29
             ...    
370470    2022-06-30
370471    2022-06-30
370472    2022-06-30
370473    2022-06-30
370474    2022-06-30
Name: caldt, Length: 8870475, dtype: object

In [21]:
age

,wficn,date,0
0,100004.0,2000-10-31,21.952538
1,100004.0,2000-11-30,21.952538
2,100004.0,2000-12-29,21.952538
3,100004.0,2001-01-31,21.952538
4,100004.0,2001-02-28,21.952538
...,...,...,...
419227,608398.0,2021-06-30,1.426450
419228,608398.0,2021-07-30,1.426450
419229,608398.0,2021-08-31,1.426450
419230,608398.0,2021-09-30,1.426450


In [126]:
returns

,wficn,date,mret,mtna,weight,rret,turnover,exp_ratio,age,cum_ret,flow,vol,obs,tna_ind
1,100004.0,2000-11-30,-0.109759,26.8,8.1,-0.109026,0.35,0.0088,21.952538,NaN,NaN,NaN,252,0
2,100004.0,2000-12-29,-0.001117,31.9,26.8,-0.000384,0.35,0.0088,21.952538,NaN,NaN,NaN,252,0
3,100004.0,2001-01-31,0.023531,40.0,31.9,0.024265,0.35,0.0088,21.952538,NaN,NaN,NaN,252,0
4,100004.0,2001-02-28,-0.118869,36.2,40.0,-0.118136,0.35,0.0088,21.952538,NaN,NaN,NaN,252,0
5,100004.0,2001-03-30,-0.088326,46.2,36.2,-0.087593,0.35,0.0088,21.952538,NaN,NaN,NaN,252,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419086,608350.0,2018-09-28,-0.010654,34.8,31.1,-0.009821,0.66,0.0100,4.501119,0.412247,NaN,NaN,48,0
419087,608350.0,2018-10-31,-0.108462,31.0,34.8,-0.107628,0.66,0.0100,4.501119,0.225598,NaN,NaN,48,0
419088,608350.0,2018-11-30,0.037101,35.5,31.0,0.037934,0.66,0.0100,4.501119,0.218804,NaN,NaN,48,0
419089,608350.0,2018-12-31,-0.081881,41.2,35.5,-0.081048,0.66,0.0100,4.501119,0.103579,NaN,0.061163,48,0


In [130]:
mret=mret.reset_index()
mret=mret[(mret['date'].astype(str)>='1980-01-01')&(mret['date'].astype(str)<'2019-02-01')]

In [131]:
mret

,wficn,date,0
0,100004.0,2000-10-31,-0.003270
1,100004.0,2000-11-30,-0.109759
2,100004.0,2000-12-29,-0.001117
3,100004.0,2001-01-31,0.023531
4,100004.0,2001-02-28,-0.118869
...,...,...,...
419086,608350.0,2018-09-28,-0.010654
419087,608350.0,2018-10-31,-0.108462
419088,608350.0,2018-11-30,0.037101
419089,608350.0,2018-12-31,-0.081881
